In [46]:
import numpy as np
import sympy as sp
from latex import *
sp.init_printing() 

In [47]:
latex_A = r"""
    
    \begin{pmatrix}
    -4 & -8 & -8\\1 & 5 & 9\\-3 & -8 & -9\\
    \end{pmatrix}
  
"""

latex_B = r"""
    
    \begin{pmatrix}
    0 & -7\\9 & 5\\-8 & -2\\
    \end{pmatrix}
  
"""

latex_lambd =r"""
    
    \begin{pmatrix}
    -6\\-6\\-7\\
    \end{pmatrix}
  
"""

latex_H = r"""
   
    \begin{pmatrix}
    1 & 0 & 0\\0 & 0 & 1\\
    \end{pmatrix}
  
"""

A = latex_to_sympy_matrix(latex_A)
B = latex_to_sympy_matrix(latex_B)
lambd = latex_to_sympy_matrix(latex_lambd)
H = latex_to_sympy_matrix(latex_H)

In [48]:
S = B.row_join(A @ B).row_join(A @ A @ B)
print("Матрица управляемости:")
print(fixed_latex(S))
S

Матрица управляемости:
\begin{pmatrix}0 & -7 & -8 & 4 & 248 & -8\\9 & 5 & -27 & 0 & -143 & -5\\-8 & -2 & 0 & -1 & 240 & -3\end{pmatrix}


⎡0   -7  -8   4   248   -8⎤
⎢                         ⎥
⎢9   5   -27  0   -143  -5⎥
⎢                         ⎥
⎣-8  -2   0   -1  240   -3⎦

Опять считается определитель неквадратной матрицы... (левый минор на самом деле, ранг просто проверяем)

In [49]:
detS = S[:, :3].det()
print("Определитель матрицы управляемости det(S) = ", detS)
print("Система управляема") if detS != 0 else print("Система не управляема (или управляема, проверьте остальные миноры :3)")

Определитель матрицы управляемости det(S) =  -1688
Система управляема


Почему тут резко начали просить в строке? А вот черт его знает

In [50]:
c = A.charpoly().all_coeffs()
c = sp.Matrix(np.flip(c[1:]).reshape(1, -1))
print("Коэффициенты характеристического полинома разомкнутой системы:")
print(fixed_latex(c))
c

Коэффициенты характеристического полинома разомкнутой системы:
\begin{pmatrix}20 & 27 & 8\end{pmatrix}


[20  27  8]

In [51]:
sigma_1 = np.sum(lambd)
sigma_2 = lambd[0] * lambd[1] + lambd[0] * lambd[2] + lambd[1] * lambd[2]
sigma_3 = np.prod(lambd)

cx = np.zeros(3, dtype=int)
cx[0] = sigma_3 * (-1) ** (3 - 0)
cx[1] = sigma_2 * (-1) ** (3 - 1)
cx[2] = sigma_1 * (-1) ** (3 - 2)

cx = sp.Matrix(cx.reshape(1, -1))

print("Коэффициенты характеристического полинома замкнутой системы:")
print(fixed_latex(cx))
cx

Коэффициенты характеристического полинома замкнутой системы:
\begin{pmatrix}252 & 120 & 19\end{pmatrix}


[252  120  19]

In [52]:
r1 = sp.symbols('r1')
r = sp.Matrix([r1, 1])
b = B @ r
d3 = b
d2 = A @ d3 + c[2] * b
d1 = A @ d2 + c[1] * b
D = np.hstack((d1, d2, d3))
D = sp.Matrix(D)
DTHT = D.T @ H.T
n2, n3 = sp.symbols('n2 n3')
LHS = sp.Matrix([[1, n2, n3]]) @ DTHT
N = sp.Matrix([1]).col_join(sp.Matrix(list(sp.solve(LHS, n2, n3).values())))
rs_dict = sp.solve((c - cx) @ N, r1)
r = r.subs(rs_dict)
print("Вектор r:")
print(fixed_latex(r))
r

Вектор r:
\begin{pmatrix}\frac{91}{760}\\1\end{pmatrix}


⎡ 91⎤
⎢───⎥
⎢760⎥
⎢   ⎥
⎣ 1 ⎦

In [53]:
DTHT = DTHT.subs(rs_dict)
print("D^T*H^T:")
print(fixed_latex(DTHT))
DTHT

D^T*H^T:
\begin{pmatrix}- \frac{13582}{95} & - \frac{5902}{95}\\- \frac{5031}{95} & - \frac{2343}{95}\\-7 & - \frac{281}{95}\end{pmatrix}


⎡-13582   -5902 ⎤
⎢───────  ──────⎥
⎢   95      95  ⎥
⎢               ⎥
⎢-5031    -2343 ⎥
⎢──────   ──────⎥
⎢  95       95  ⎥
⎢               ⎥
⎢         -281  ⎥
⎢  -7     ───── ⎥
⎣           95  ⎦

In [54]:
q1, q2 = sp.symbols('q1 q2')
V = sp.Matrix([q1, q2])
V = sp.Matrix(list(sp.solve(DTHT @ V - (c - cx).T, (q1, q2)).values())).T
print("Матрица V:")
print(fixed_latex(V))
V

Матрица V:
\begin{pmatrix}- \frac{1425}{6016} & \frac{25745}{6016}\end{pmatrix}


⎡-1425   25745⎤
⎢──────  ─────⎥
⎣ 6016    6016⎦

In [55]:
P = r @ V
print("Матрица управления P")
print(fixed_latex(P))
P

Матрица управления P
\begin{pmatrix}- \frac{1365}{48128} & \frac{24661}{48128}\\- \frac{1425}{6016} & \frac{25745}{6016}\end{pmatrix}


⎡-1365   24661⎤
⎢──────  ─────⎥
⎢48128   48128⎥
⎢             ⎥
⎢-1425   25745⎥
⎢──────  ─────⎥
⎣ 6016    6016⎦